In [2]:
import onnxruntime
import numpy as np
import torch
session = onnxruntime.InferenceSession("/datadrive/tensorrt/onnx_models/mobilenetv2-12.onnx",providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider'])
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

2022-07-04 07:24:11.361218177 [W:onnxruntime:Default, tensorrt_execution_provider.h:59 log] [2022-07-04 07:24:11 WARNING] external/onnx-tensorrt/onnx2trt_utils.cpp:364: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.


In [3]:
session

In [4]:
session.get_modelmeta()


In [5]:
len(session.get_inputs()[0].shape)

4

In [6]:
session.get_inputs()[0].shape

['batch_size', 3, 224, 224]

In [7]:
session.get_inputs()[0].shape[0]

'batch_size'

In [8]:
session.get_inputs()[0].shape[1]

3

In [9]:
session.get_inputs()[0].shape[2]

224

In [10]:
session.get_inputs()[0].shape[3]

224

In [11]:
session.get_outputs()[0].shape

['batch_size', 1000]

In [13]:
session.get_modelmeta()
first_input_name = session.get_inputs()[0].name
first_output_name = session.get_outputs()[0].name

In [14]:
first_input_name

'input'

In [15]:
first_output_name

'output'

In [16]:
indata = torch.randn(1,3, 224, 224).to(device)

In [17]:
indata
X=torch.randn(3, 224, 224)
print(X)

tensor([[[-1.3997,  0.4702, -0.6243,  ..., -1.3971, -1.6156, -0.1182],
         [-1.2046,  1.4137,  0.1384,  ..., -0.7232,  0.0760, -0.5976],
         [-0.7063,  2.3775,  1.1319,  ...,  1.5302,  0.9688,  0.8922],
         ...,
         [ 0.1220, -0.2061,  0.2736,  ..., -1.0567, -1.4441, -0.2421],
         [-1.2406,  1.1382,  0.0758,  ...,  0.8747,  0.4169,  0.5828],
         [-0.2494, -0.3810,  0.0665,  ..., -0.3015,  0.5078, -1.7906]],

        [[-0.0177, -0.1914,  1.6661,  ...,  0.1045, -0.7960,  0.9976],
         [-2.8699, -0.6795,  0.0402,  ..., -1.3864, -0.6110, -0.0958],
         [ 1.7560, -1.4383, -0.5974,  ...,  0.1397, -0.0680, -1.2598],
         ...,
         [-0.4202,  0.2428, -0.1743,  ...,  0.8596,  0.6312,  0.1220],
         [-0.4560,  1.5947, -0.3342,  ...,  0.0631, -0.8836,  1.8298],
         [-0.5669, -1.2307, -0.4957,  ...,  0.9746,  0.2880, -0.3170]],

        [[ 0.3490,  1.1921,  1.3749,  ..., -0.2388, -1.6298,  2.7641],
         [-0.3295, -0.1010,  0.1231,  ..., -0

In [18]:
# results = session.run(["output1", "output2"], {
#                       "input1": indata1, "input2": indata2})

results = session.run(None, {'input':to_numpy(indata)})
results

NameError: name 'to_numpy' is not defined

In [ ]:
import onnx

onnx_model = onnx.load("/datadrive/tensorrt/onnx_models/mobilenetv2-12.onnx")
onnx.checker.check_model(onnx_model)

In [21]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("/datadrive/tensorrt/onnx_models/mobilenetv2-12.onnx",providers=['CUDAExecutionProvider','TensorrtExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
x = torch.randn(1,3, 224, 224).to(device)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

#print("Exported model has been tested with ONNXRuntime, and the result looks good!")
len(ort_outs[0][0])
print(ort_outs[0])
np.array(ort_outs[0]).shape

[[-2.3513658e+00  3.4880078e-01  1.0479385e+00  5.6619471e-01
   1.7580612e+00  3.2356071e+00  2.1380420e+00 -3.4395054e-01
  -3.1979823e-01 -1.6917720e+00 -8.8819480e-01  1.0177817e+00
   1.2250296e+00  6.9235086e-01  4.9656609e-01  1.7352149e+00
   1.0880485e+00  8.3407968e-01  3.7404914e+00  4.0458649e-02
   1.4404637e-01 -2.8354841e-01 -1.6670184e-01  4.0562540e-01
  -5.3319895e-01 -1.4196340e+00 -1.4434697e-01  6.1067307e-01
  -8.5215694e-01 -1.5076911e+00 -1.5236316e+00  1.0633401e+00
  -2.5840645e+00 -7.9015243e-01  3.5722625e-01 -2.1516514e+00
  -4.4464308e-01 -7.9954788e-02  1.3693851e-01 -5.4282832e-01
   9.7520709e-01 -8.5004735e-01  8.4889489e-01 -1.4219348e+00
   5.3716439e-01 -9.4955403e-01  1.9042561e+00 -2.7461679e+00
  -8.0272317e-01 -2.5872738e+00 -6.1017156e-01 -2.7433674e+00
   7.3806828e-01 -3.5241678e-01 -2.9744831e-01 -2.6853490e-01
  -1.1747402e+00 -2.9060104e+00 -1.3134469e-01 -4.2644808e-01
   5.1081467e-01 -2.5085018e+00 -1.8930715e+00 -2.1359186e+00
  -1.057

(1, 1000)

In [24]:
print(ort_outs[0].shape)

(1, 1000)


In [ ]:
batch_size = 1 if input_shape[0] == 'batch_size' else 1

In [26]:
onnxruntime.get_available_providers()

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']